In [22]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [23]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/paolamateo/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


## Visit the NASA mars news site

In [24]:
# Visit the Mars news site
url = 'https://redplanetscience.com/'
browser.visit(url)

soup=bs(browser.html,'html.parser')

title=soup.find('div', class_="content_title").text
para=soup.find('div', class_="article_teaser_body").text

## JPL Space Images Featured Image

In [23]:
#Mars Featured Image
url= "https://spaceimages-mars.com"
browser.visit(url)
soup = bs(browser.html,'html.parser')

In [24]:
truncated_url=soup.find('img', class_="headerimage")["src"]
featured_img_url=f"{url}/{truncated_url}"

<div class="content_title">NASA Adds Return Sample Scientists to Mars 2020 Leadership Team</div>

## Mars Facts

In [25]:
#Mars Facts
url= "https://galaxyfacts-mars.com"

'NASA Adds Return Sample Scientists to Mars 2020 Leadership Team'

In [26]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=df.iloc[0]
df=df[1:]
df.set_index(df.columns[0])
td= df.to_html()
td

'The leadership council for Mars 2020 science added two new members who represent the interests of scientists destined to handle and study the first samples from Red Planet.'

## Hemispheres

In [25]:
url = 'https://marshemispheres.com/'

browser.visit(url)
soup=bs(browser.html, 'html.parser')

In [26]:
h3=soup.find_all('h3')
four_h3= [tag.text for tag in h3][:-1]

In [27]:
# Create a list to hold the images and titles.
hemisphere_image_urls = []

# Get a list of all of the hemispheres
links = browser.find_by_css('a.product-item img')

# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css('a.product-item img')[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css('h2.title').text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()

In [28]:
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [29]:
browser.quit()